# Ambiente

In [1]:
!apt-get install default-jre
!java -version
!pip install h2o
import h2o
from h2o.automl import H2OAutoML
h2o.init()

import pandas as pd
from datetime import datetime

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
openjdk version "11.0.15" 2022-04-19
OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1)
OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
     |████████████████████████████████| 177.0 MB 13 kB/s 
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=3fc6fd2bb25e0db66864d7b37bfae06d0b45531d61bf1ec3ccc274afa3aa7347
  Stored in directory: /root/.cache/pip/wheels/a6/d9/ab/5442447c7e2ccf07f66aa8b79f3877ce5382f0b95e6c0c797b
Successfully built h2o
Checking whether there is a

H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,24 days
H2O_cluster_name:,H2O_from_python_unknownUser_iitc18
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


# H2O

In [7]:
tempo_processamento_sec = 60*60
seed = 123
balance_classes = False

## Adult.csv

In [ ]:
dataset = "adult"
url = '/content/drive/MyDrive/Luiz Gabriel - PC/13.Pós Data Science UFPR/00.TCC/Especializacao_Data_Science_Big_Data_Monografia/data/adult.csv'
df = pd.read_csv(url, sep=',')
hf = h2o.H2OFrame(df)
#hf.describe(chunk_summary=True)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
start_time = datetime.now()

# Separa base de treino e teste
train, test = hf.split_frame(ratios=[0.8], seed = 1)

# Editing pycaret's setup
aml = H2OAutoML(max_runtime_secs = tempo_processamento_sec,
                balance_classes=balance_classes,
                seed = seed)

# Compare models and choose de best by AUC
aml.train(training_frame = train, y = 'class')

# Tuning best model hyperparameters
lb = aml.leaderboard
df_lb = lb.as_data_frame()

# Filtering Ensemble models
metrics = df_lb[~df_lb['model_id'].str.contains("Ensemble")].head(1)

end_time = datetime.now()
execution_time = end_time - start_time
print('Duration: {}'.format(end_time - start_time))

# Getting metrics
metrics["execution_time"] = execution_time
metrics["dataset"] = dataset

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%

18:53:26.487: GBM_lr_annealing_selection_AutoML_1_20220501_175337 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

Duration: 1:00:04.419859


In [ ]:
# Create model_metrics dataframe to store all the experiments metrics
models_metrics = metrics.copy()

del train, test, aml, lb, df_lb, metrics

## kc1.csv

In [ ]:
dataset = "kc1"
url = '/content/drive/MyDrive/Luiz Gabriel - PC/13.Pós Data Science UFPR/00.TCC/Especializacao_Data_Science_Big_Data_Monografia/data/kc1.csv'
df = pd.read_csv(url, sep=',')
hf = h2o.H2OFrame(df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
start_time = datetime.now()

train, test = hf.split_frame(ratios=[0.8], seed = 1)

# Editing pycaret's setup
aml = H2OAutoML(max_runtime_secs = tempo_processamento_sec,
                balance_classes=balance_classes,
                seed = seed)

# Compare models and choose de best by AUC
aml.train(training_frame = train, y = 'defects')

# Tuning best model hyperparameters
lb = aml.leaderboard
df_lb = lb.as_data_frame()
metrics = df_lb[~df_lb['model_id'].str.contains("Ensemble")].head(1)

end_time = datetime.now()
execution_time = end_time - start_time
print('Duration: {}'.format(end_time - start_time))

# Getting metrics
metrics["execution_time"] = execution_time
metrics["dataset"] = dataset

models_metrics = models_metrics.append(metrics)


del aml, lb, df_lb, metrics

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Duration: 0:59:04.188073


In [ ]:
models_metrics

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,execution_time,dataset
5,GBM_grid_1_AutoML_1_20220501_175337_model_30,0.929071,0.276896,0.829753,0.170606,0.295700,0.087439,0 days 01:00:04.419859,adult
9,GBM_grid_1_AutoML_2_20220501_185341_model_42,0.798614,0.348031,0.464187,0.285142,0.326486,0.106593,0 days 00:59:04.188073,kc1


## blood_transfusion.csv

In [4]:
dataset = "blood_transfusion"
url = '/content/drive/MyDrive/Luiz Gabriel - PC/13.Pós Data Science UFPR/00.TCC/Especializacao_Data_Science_Big_Data_Monografia/data/blood_transfusion.csv'
df = pd.read_csv(url, sep=',')
hf = h2o.H2OFrame(df)
hf["Class"] = hf["Class"].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
start_time = datetime.now()

train, test = hf.split_frame(ratios=[0.8], seed = 1)

# Editing pycaret's setup
aml = H2OAutoML(max_runtime_secs = tempo_processamento_sec,
                balance_classes=balance_classes,
                seed = seed)

# Compare models and choose de best by AUC
aml.train(training_frame = train, y = 'Class')

# Tuning best model hyperparameters
lb = aml.leaderboard
df_lb = lb.as_data_frame()
df_lb = df_lb[~df_lb['model_id'].str.contains("Deep")]
metrics = df_lb[~df_lb['model_id'].str.contains("Ensemble")].head(1)

end_time = datetime.now()
execution_time = end_time - start_time
print('Duration: {}'.format(end_time - start_time))

# Getting metrics
metrics["execution_time"] = execution_time
metrics["dataset"] = dataset

#models_metrics = models_metrics.append(metrics)


#del aml, lb, df_lb, metrics

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Duration: 0:57:14.760303


In [10]:
metrics

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,execution_time,dataset
19,XGBoost_lr_search_selection_AutoML_2_20220508_...,0.734961,0.494644,0.46822,0.311593,0.400999,0.1608,0 days 00:57:14.760303,blood_transfusion


In [ ]:
models_metrics

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,execution_time,dataset
5,GBM_grid_1_AutoML_1_20220501_175337_model_30,0.929071,0.276896,0.829753,0.170606,0.295700,0.087439,0 days 01:00:04.419859,adult
9,GBM_grid_1_AutoML_2_20220501_185341_model_42,0.798614,0.348031,0.464187,0.285142,0.326486,0.106593,0 days 00:59:04.188073,kc1
0,DeepLearning_grid_1_AutoML_3_20220501_195245_m...,0.749150,0.490655,0.484280,0.293630,0.399378,0.159503,0 days 00:57:11.442520,blood_transfusion


## nomao

In [ ]:
dataset = "nomao"
url = '/content/drive/MyDrive/Luiz Gabriel - PC/13.Pós Data Science UFPR/00.TCC/Especializacao_Data_Science_Big_Data_Monografia/data/nomao.csv'
df = pd.read_csv(url, sep=',')
hf = h2o.H2OFrame(df)
hf["Class"] = hf["Class"].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
start_time = datetime.now()

train, test = hf.split_frame(ratios=[0.8], seed = 1)

# Editing pycaret's setup
aml = H2OAutoML(max_runtime_secs = tempo_processamento_sec,
                balance_classes=balance_classes,
                seed = seed)

# Compare models and choose de best by AUC
aml.train(training_frame = train, y = 'Class')

# Tuning best model hyperparameters
lb = aml.leaderboard
df_lb = lb.as_data_frame()
metrics = df_lb[~df_lb['model_id'].str.contains("Ensemble")].head(1)

end_time = datetime.now()
execution_time = end_time - start_time
print('Duration: {}'.format(end_time - start_time))

# Getting metrics
metrics["execution_time"] = execution_time
metrics["dataset"] = dataset

models_metrics = models_metrics.append(metrics)


del aml, lb, df_lb, metrics


AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Duration: 1:00:05.910690


In [ ]:
models_metrics

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,execution_time,dataset
5,GBM_grid_1_AutoML_1_20220501_175337_model_30,0.929071,0.276896,0.829753,0.170606,0.295700,0.087439,0 days 01:00:04.419859,adult
9,GBM_grid_1_AutoML_2_20220501_185341_model_42,0.798614,0.348031,0.464187,0.285142,0.326486,0.106593,0 days 00:59:04.188073,kc1
0,DeepLearning_grid_1_AutoML_3_20220501_195245_m...,0.749150,0.490655,0.484280,0.293630,0.399378,0.159503,0 days 00:57:11.442520,blood_transfusion
6,GBM_grid_1_AutoML_4_20220501_205006_model_2,0.995724,0.073501,0.998259,0.033596,0.144285,0.020818,0 days 01:00:05.910690,nomao


## kdd

In [ ]:
dataset = "kdd"
url = '/content/drive/MyDrive/Luiz Gabriel - PC/13.Pós Data Science UFPR/00.TCC/Especializacao_Data_Science_Big_Data_Monografia/data/kdd.csv'
df = pd.read_csv(url, sep=',')
hf = h2o.H2OFrame(df)
hf["APPETENCY"] = hf["APPETENCY"].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
start_time = datetime.now()

train, test = hf.split_frame(ratios=[0.8], seed = 1)

# Editing pycaret's setup
aml = H2OAutoML(max_runtime_secs = tempo_processamento_sec,
                balance_classes=balance_classes,
                seed = seed)

# Compare models and choose de best by AUC
aml.train(training_frame = train, y = 'APPETENCY')

# Tuning best model hyperparameters
lb = aml.leaderboard
df_lb = lb.as_data_frame()
metrics = df_lb[~df_lb['model_id'].str.contains("Ensemble")].head(1)

end_time = datetime.now()
execution_time = end_time - start_time
print('Duration: {}'.format(end_time - start_time))

# Getting metrics
metrics["execution_time"] = execution_time
metrics["dataset"] = dataset

models_metrics = models_metrics.append(metrics)


del aml, lb, df_lb, metrics


AutoML progress: |
21:50:36.359: _train param, Dropping bad and constant columns: [Var79, Var169, Var15, Var48, Var39, Var175, Var230, Var167, Var141, Var185, Var20, Var31, Var42, Var52, Var55, Var32, Var209, Var8]

████
21:53:00.706: _train param, Dropping bad and constant columns: [Var79, Var169, Var15, Var48, Var138, Var39, Var175, Var230, Var167, Var141, Var185, Var20, Var31, Var42, Var52, Var55, Var32, Var209, Var8]

█████
21:58:16.858: _train param, Dropping bad and constant columns: [Var79, Var169, Var15, Var48, Var39, Var175, Var230, Var167, Var141, Var185, Var20, Var31, Var42, Var52, Var55, Var32, Var209, Var8]

███
22:00:48.73: _train param, Dropping unused columns: [Var79, Var169, Var15, Var48, Var39, Var175, Var230, Var167, Var141, Var185, Var20, Var31, Var42, Var52, Var55, Var32, Var209, Var8]


22:00:52.872: _train param, Dropping bad and constant columns: [Var79, Var169, Var15, Var48, Var39, Var175, Var230, Var167, Var141, Var185, Var20, Var31, Var42, Var52, Var55, Var32

In [ ]:
models_metrics

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,execution_time,dataset
5,GBM_grid_1_AutoML_1_20220501_175337_model_30,0.929071,0.276896,0.829753,0.170606,0.295700,0.087439,0 days 01:00:04.419859,adult
9,GBM_grid_1_AutoML_2_20220501_185341_model_42,0.798614,0.348031,0.464187,0.285142,0.326486,0.106593,0 days 00:59:04.188073,kc1
0,DeepLearning_grid_1_AutoML_3_20220501_195245_m...,0.749150,0.490655,0.484280,0.293630,0.399378,0.159503,0 days 00:57:11.442520,blood_transfusion
6,GBM_grid_1_AutoML_4_20220501_205006_model_2,0.995724,0.073501,0.998259,0.033596,0.144285,0.020818,0 days 01:00:05.910690,nomao
5,GLM_1_AutoML_5_20220501_215036,0.818971,0.077443,0.083362,0.374588,0.130687,0.017079,0 days 01:00:06.112315,kdd


## bank_marketing

In [ ]:
dataset = "bank_marketing"
url = '/content/drive/MyDrive/Luiz Gabriel - PC/13.Pós Data Science UFPR/00.TCC/Especializacao_Data_Science_Big_Data_Monografia/data/bank_marketing.csv'
df = pd.read_csv(url, sep=',')
hf = h2o.H2OFrame(df)
hf["Class"] = hf["Class"].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
start_time = datetime.now()

train, test = hf.split_frame(ratios=[0.8], seed = 1)

# Editing pycaret's setup
aml = H2OAutoML(max_runtime_secs = tempo_processamento_sec,
                balance_classes=balance_classes,
                seed = seed)

# Compare models and choose de best by AUC
aml.train(training_frame = train, y = 'Class')

# Tuning best model hyperparameters
lb = aml.leaderboard
df_lb = lb.as_data_frame()
metrics = df_lb[~df_lb['model_id'].str.contains("Ensemble")].head(1)

end_time = datetime.now()
execution_time = end_time - start_time
print('Duration: {}'.format(end_time - start_time))

# Getting metrics
metrics["execution_time"] = execution_time
metrics["dataset"] = dataset

models_metrics = models_metrics.append(metrics)


del aml, lb, df_lb, metrics


AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%

23:50:34.233: GBM_lr_annealing_selection_AutoML_6_20220501_225041 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

Duration: 1:00:02.735565


In [ ]:
models_metrics

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,execution_time,dataset
5,GBM_grid_1_AutoML_1_20220501_175337_model_30,0.929071,0.276896,0.829753,0.170606,0.295700,0.087439,0 days 01:00:04.419859,adult
9,GBM_grid_1_AutoML_2_20220501_185341_model_42,0.798614,0.348031,0.464187,0.285142,0.326486,0.106593,0 days 00:59:04.188073,kc1
0,DeepLearning_grid_1_AutoML_3_20220501_195245_m...,0.749150,0.490655,0.484280,0.293630,0.399378,0.159503,0 days 00:57:11.442520,blood_transfusion
6,GBM_grid_1_AutoML_4_20220501_205006_model_2,0.995724,0.073501,0.998259,0.033596,0.144285,0.020818,0 days 01:00:05.910690,nomao
5,GLM_1_AutoML_5_20220501_215036,0.818971,0.077443,0.083362,0.374588,0.130687,0.017079,0 days 01:00:06.112315,kdd
8,GBM_1_AutoML_6_20220501_225041,0.933190,0.198512,0.615708,0.170414,0.250423,0.062711,0 days 01:00:02.735565,bank_marketing


## amazon_emp

In [ ]:
dataset = "amazon_emp"
url = '/content/drive/MyDrive/Luiz Gabriel - PC/13.Pós Data Science UFPR/00.TCC/Especializacao_Data_Science_Big_Data_Monografia/data/amazon_emp.csv'
df = pd.read_csv(url, sep=',')
hf = h2o.H2OFrame(df)
hf["target"] = hf["target"].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
start_time = datetime.now()

train, test = hf.split_frame(ratios=[0.8], seed = 1)

# Editing pycaret's setup
aml = H2OAutoML(max_runtime_secs = tempo_processamento_sec,
                balance_classes=balance_classes,
                seed = seed)

# Compare models and choose de best by AUC
aml.train(training_frame = train, y = 'target')

# Tuning best model hyperparameters
lb = aml.leaderboard
df_lb = lb.as_data_frame()
metrics = df_lb[~df_lb['model_id'].str.contains("Ensemble")].head(1)

end_time = datetime.now()
execution_time = end_time - start_time
print('Duration: {}'.format(end_time - start_time))

# Getting metrics
metrics["execution_time"] = execution_time
metrics["dataset"] = dataset

models_metrics = models_metrics.append(metrics)


del aml, lb, df_lb, metrics


AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%

00:50:41.143: GBM_lr_annealing_selection_AutoML_7_20220501_235045 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

Duration: 1:00:03.378219


In [ ]:
models_metrics

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,execution_time,dataset
5,GBM_grid_1_AutoML_1_20220501_175337_model_30,0.929071,0.276896,0.829753,0.170606,0.295700,0.087439,0 days 01:00:04.419859,adult
9,GBM_grid_1_AutoML_2_20220501_185341_model_42,0.798614,0.348031,0.464187,0.285142,0.326486,0.106593,0 days 00:59:04.188073,kc1
0,DeepLearning_grid_1_AutoML_3_20220501_195245_m...,0.749150,0.490655,0.484280,0.293630,0.399378,0.159503,0 days 00:57:11.442520,blood_transfusion
6,GBM_grid_1_AutoML_4_20220501_205006_model_2,0.995724,0.073501,0.998259,0.033596,0.144285,0.020818,0 days 01:00:05.910690,nomao
5,GLM_1_AutoML_5_20220501_215036,0.818971,0.077443,0.083362,0.374588,0.130687,0.017079,0 days 01:00:06.112315,kdd
8,GBM_1_AutoML_6_20220501_225041,0.933190,0.198512,0.615708,0.170414,0.250423,0.062711,0 days 01:00:02.735565,bank_marketing
4,GBM_grid_1_AutoML_7_20220501_235045_model_15,0.848280,0.161044,0.984962,0.372670,0.204500,0.041820,0 days 01:00:03.378219,amazon_emp


## jasmine

In [ ]:
dataset = "jasmine"
url = '/content/drive/MyDrive/Luiz Gabriel - PC/13.Pós Data Science UFPR/00.TCC/Especializacao_Data_Science_Big_Data_Monografia/data/jasmine.csv'
df = pd.read_csv(url, sep=',')
hf = h2o.H2OFrame(df)
hf["class"] = hf["class"].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
start_time = datetime.now()

train, test = hf.split_frame(ratios=[0.8], seed = 1)

# Editing pycaret's setup
aml = H2OAutoML(max_runtime_secs = tempo_processamento_sec,
                balance_classes=balance_classes,
                seed = seed)

# Compare models and choose de best by AUC
aml.train(training_frame = train, y = 'class')

# Tuning best model hyperparameters
lb = aml.leaderboard
df_lb = lb.as_data_frame()
metrics = df_lb[~df_lb['model_id'].str.contains("Ensemble")].head(1)

end_time = datetime.now()
execution_time = end_time - start_time
print('Duration: {}'.format(end_time - start_time))

# Getting metrics
metrics["execution_time"] = execution_time
metrics["dataset"] = dataset

models_metrics = models_metrics.append(metrics)


del aml, lb, df_lb, metrics


AutoML progress: |██████████████████████████████████████████████████████████████
01:48:08.863: GBM_lr_annealing_selection_AutoML_8_20220502_05049 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

█| (done) 100%
Duration: 0:59:26.203193


In [ ]:
models_metrics

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,execution_time,dataset
5,GBM_grid_1_AutoML_1_20220501_175337_model_30,0.929071,0.276896,0.829753,0.170606,0.295700,0.087439,0 days 01:00:04.419859,adult
9,GBM_grid_1_AutoML_2_20220501_185341_model_42,0.798614,0.348031,0.464187,0.285142,0.326486,0.106593,0 days 00:59:04.188073,kc1
0,DeepLearning_grid_1_AutoML_3_20220501_195245_m...,0.749150,0.490655,0.484280,0.293630,0.399378,0.159503,0 days 00:57:11.442520,blood_transfusion
6,GBM_grid_1_AutoML_4_20220501_205006_model_2,0.995724,0.073501,0.998259,0.033596,0.144285,0.020818,0 days 01:00:05.910690,nomao
5,GLM_1_AutoML_5_20220501_215036,0.818971,0.077443,0.083362,0.374588,0.130687,0.017079,0 days 01:00:06.112315,kdd
8,GBM_1_AutoML_6_20220501_225041,0.933190,0.198512,0.615708,0.170414,0.250423,0.062711,0 days 01:00:02.735565,bank_marketing
4,GBM_grid_1_AutoML_7_20220501_235045_model_15,0.848280,0.161044,0.984962,0.372670,0.204500,0.041820,0 days 01:00:03.378219,amazon_emp
5,GBM_grid_1_AutoML_8_20220502_05049_model_7,0.879640,0.402137,0.831079,0.185963,0.362368,0.131311,0 days 00:59:26.203193,jasmine


## kr_vs_kp

In [ ]:
dataset = "kr_vs_kp"
url = '/content/drive/MyDrive/Luiz Gabriel - PC/13.Pós Data Science UFPR/00.TCC/Especializacao_Data_Science_Big_Data_Monografia/data/kr_vs_kp.csv'
df = pd.read_csv(url, sep=',')
hf = h2o.H2OFrame(df)
hf["class"] = hf["class"].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
start_time = datetime.now()

train, test = hf.split_frame(ratios=[0.8], seed = 1)

# Editing pycaret's setup
aml = H2OAutoML(max_runtime_secs = tempo_processamento_sec,
                balance_classes=balance_classes,
                seed = seed)

# Compare models and choose de best by AUC
aml.train(training_frame = train, y = 'class')

# Tuning best model hyperparameters
lb = aml.leaderboard
df_lb = lb.as_data_frame()
metrics = df_lb[~df_lb['model_id'].str.contains("Ensemble")].head(1)

end_time = datetime.now()
execution_time = end_time - start_time
print('Duration: {}'.format(end_time - start_time))

# Getting metrics
metrics["execution_time"] = execution_time
metrics["dataset"] = dataset

models_metrics = models_metrics.append(metrics)


del aml, lb, df_lb, metrics

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%

02:49:43.643: GBM_lr_annealing_selection_AutoML_9_20220502_15015 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

Duration: 1:00:07.030194


In [ ]:
models_metrics

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,execution_time,dataset
5,GBM_grid_1_AutoML_1_20220501_175337_model_30,0.929071,0.276896,0.829753,0.170606,0.295700,0.087439,0 days 01:00:04.419859,adult
9,GBM_grid_1_AutoML_2_20220501_185341_model_42,0.798614,0.348031,0.464187,0.285142,0.326486,0.106593,0 days 00:59:04.188073,kc1
0,DeepLearning_grid_1_AutoML_3_20220501_195245_m...,0.749150,0.490655,0.484280,0.293630,0.399378,0.159503,0 days 00:57:11.442520,blood_transfusion
6,GBM_grid_1_AutoML_4_20220501_205006_model_2,0.995724,0.073501,0.998259,0.033596,0.144285,0.020818,0 days 01:00:05.910690,nomao
5,GLM_1_AutoML_5_20220501_215036,0.818971,0.077443,0.083362,0.374588,0.130687,0.017079,0 days 01:00:06.112315,kdd
8,GBM_1_AutoML_6_20220501_225041,0.933190,0.198512,0.615708,0.170414,0.250423,0.062711,0 days 01:00:02.735565,bank_marketing
4,GBM_grid_1_AutoML_7_20220501_235045_model_15,0.848280,0.161044,0.984962,0.372670,0.204500,0.041820,0 days 01:00:03.378219,amazon_emp
5,GBM_grid_1_AutoML_8_20220502_05049_model_7,0.879640,0.402137,0.831079,0.185963,0.362368,0.131311,0 days 00:59:26.203193,jasmine
1,GBM_grid_1_AutoML_9_20220502_15015_model_5,0.999743,0.013693,0.999748,0.002772,0.053545,0.002867,0 days 01:00:07.030194,kr_vs_kp


## higgs

In [ ]:
dataset = "higgs"
url = '/content/drive/MyDrive/Luiz Gabriel - PC/13.Pós Data Science UFPR/00.TCC/Especializacao_Data_Science_Big_Data_Monografia/data/higgs.csv'
df = pd.read_csv(url, sep=',')
hf = h2o.H2OFrame(df)
hf["class"] = hf["class"].asfactor()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (20,21,22,23,24,25,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
start_time = datetime.now()

train, test = hf.split_frame(ratios=[0.8], seed = 1)

# Editing pycaret's setup
aml = H2OAutoML(max_runtime_secs = tempo_processamento_sec,
                balance_classes=balance_classes,
                seed = seed)

# Compare models and choose de best by AUC
aml.train(training_frame = train, y = 'class')

# Tuning best model hyperparameters
lb = aml.leaderboard
df_lb = lb.as_data_frame()
metrics = df_lb[~df_lb['model_id'].str.contains("Ensemble")].head(1)

end_time = datetime.now()
execution_time = end_time - start_time
print('Duration: {}'.format(end_time - start_time))

# Getting metrics
metrics["execution_time"] = execution_time
metrics["dataset"] = dataset

models_metrics = models_metrics.append(metrics)


del aml, lb, df_lb, metrics

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Duration: 1:00:06.855815


In [ ]:
models_metrics

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,execution_time,dataset
5,GBM_grid_1_AutoML_1_20220501_175337_model_30,0.929071,0.276896,0.829753,0.170606,0.295700,0.087439,0 days 01:00:04.419859,adult
9,GBM_grid_1_AutoML_2_20220501_185341_model_42,0.798614,0.348031,0.464187,0.285142,0.326486,0.106593,0 days 00:59:04.188073,kc1
0,DeepLearning_grid_1_AutoML_3_20220501_195245_m...,0.749150,0.490655,0.484280,0.293630,0.399378,0.159503,0 days 00:57:11.442520,blood_transfusion
6,GBM_grid_1_AutoML_4_20220501_205006_model_2,0.995724,0.073501,0.998259,0.033596,0.144285,0.020818,0 days 01:00:05.910690,nomao
5,GLM_1_AutoML_5_20220501_215036,0.818971,0.077443,0.083362,0.374588,0.130687,0.017079,0 days 01:00:06.112315,kdd
8,GBM_1_AutoML_6_20220501_225041,0.933190,0.198512,0.615708,0.170414,0.250423,0.062711,0 days 01:00:02.735565,bank_marketing
4,GBM_grid_1_AutoML_7_20220501_235045_model_15,0.848280,0.161044,0.984962,0.372670,0.204500,0.041820,0 days 01:00:03.378219,amazon_emp
5,GBM_grid_1_AutoML_8_20220502_05049_model_7,0.879640,0.402137,0.831079,0.185963,0.362368,0.131311,0 days 00:59:26.203193,jasmine
1,GBM_grid_1_AutoML_9_20220502_15015_model_5,0.999743,0.013693,0.999748,0.002772,0.053545,0.002867,0 days 01:00:07.030194,kr_vs_kp
8,GBM_1_AutoML_10_20220502_25031,0.804615,0.533962,0.821308,0.308424,0.424143,0.179898,0 days 01:00:06.855815,higgs


# Resultados

In [ ]:
models_metrics

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,execution_time,dataset
5,GBM_grid_1_AutoML_1_20220501_175337_model_30,0.929071,0.276896,0.829753,0.170606,0.295700,0.087439,0 days 01:00:04.419859,adult
9,GBM_grid_1_AutoML_2_20220501_185341_model_42,0.798614,0.348031,0.464187,0.285142,0.326486,0.106593,0 days 00:59:04.188073,kc1
0,DeepLearning_grid_1_AutoML_3_20220501_195245_m...,0.749150,0.490655,0.484280,0.293630,0.399378,0.159503,0 days 00:57:11.442520,blood_transfusion
6,GBM_grid_1_AutoML_4_20220501_205006_model_2,0.995724,0.073501,0.998259,0.033596,0.144285,0.020818,0 days 01:00:05.910690,nomao
5,GLM_1_AutoML_5_20220501_215036,0.818971,0.077443,0.083362,0.374588,0.130687,0.017079,0 days 01:00:06.112315,kdd
8,GBM_1_AutoML_6_20220501_225041,0.933190,0.198512,0.615708,0.170414,0.250423,0.062711,0 days 01:00:02.735565,bank_marketing
4,GBM_grid_1_AutoML_7_20220501_235045_model_15,0.848280,0.161044,0.984962,0.372670,0.204500,0.041820,0 days 01:00:03.378219,amazon_emp
5,GBM_grid_1_AutoML_8_20220502_05049_model_7,0.879640,0.402137,0.831079,0.185963,0.362368,0.131311,0 days 00:59:26.203193,jasmine
1,GBM_grid_1_AutoML_9_20220502_15015_model_5,0.999743,0.013693,0.999748,0.002772,0.053545,0.002867,0 days 01:00:07.030194,kr_vs_kp
8,GBM_1_AutoML_10_20220502_25031,0.804615,0.533962,0.821308,0.308424,0.424143,0.179898,0 days 01:00:06.855815,higgs


In [ ]:
from google.colab import files

models_metrics.to_csv('mestric_h20.csv', encoding = 'utf-8-sig') 
files.download('mestric_h20.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>